# Introdução

Trata-se de uma análise de dados a partir de uma amostra de base de dados providenciada pela empresa Doc9, "responsável por enviar advogados e prepostos para representar grandes clientes em audiências em todo o território nacional".

A seguir serão apresentados, em sua melhor capacidade, tentativas de responder perguntas feitas por interessados no negócio.

Será considerado que a amostra de dados providenciada representa uma _amostra estatisticamente significativa_ daquela que a originou.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Em razão da estrutura da árvore de arquivos é interessante utilizar a
# biblioteca pathlib para criar um caminho mais conveniente para encontrar
# os dados a serem analisados

data = Path.cwd().parent / 'data'

In [3]:
df = pd.read_csv(
    data / 'solicitacoes_doc9.csv',
    parse_dates=[
        'datahora_abertura_solicitacao',
        'datahora_finalizacao_solicitacao',
        'datahora_audiencia',
        'prazo_para_inserir_dados',
        ],
    sep=';'
    )

# Exclui coluna provavelmente proveniente
# do índice do banco de dados original
df = df.drop(columns=['Unnamed: 0'])

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   id_solicitacao                    30000 non-null  int64         
 1   nome_parceiro                     30000 non-null  object        
 2   nome_cliente                      30000 non-null  object        
 3   datahora_abertura_solicitacao     30000 non-null  datetime64[ns]
 4   datahora_audiencia                30000 non-null  datetime64[ns]
 5   prazo_para_inserir_dados          30000 non-null  datetime64[ns]
 6   datahora_finalizacao_solicitacao  29933 non-null  datetime64[ns]
 7   tipo                              30000 non-null  object        
 8   tipo_demanda                      30000 non-null  object        
 9   area_processo                     29999 non-null  object        
 10  tipo_audiencia                    30000 non-nu

As seguintes colunas contém valores nulos:

| Nome da coluna                   | Valores válidos |
|----------------------------------|-----------------|
| datahora_finalizacao_solicitacao |           29933 |
| area_processo                    |           29999 |
| comarca                          |           29992 |
| situacao_dados                   |           29998 |

## Tratamento de valores nulos

Alguns algoritmos de machine learning não respondem bem ao treinamento com valores nulos, impactando seu resultado.

Este é um bom momento, portanto, para tratar estes dados e evitar _bugs_ no futuro que poderão reduzir a performance das análises e, consequetemente, reduzir a qualidade dos insights gerados.

### datahora_finalizacao_solicitacao

Métodos mais comuns para preencher dados nulos é utilizar a média ou a moda da coluna, porém este não me parece ser o melhor caminho neste campo específico.

Dependendo do processo podemos gerar eventos como solicitações que foram finalizadas antes mesmo de serem solicitadas.

Para contornar esta realidade, uma possível solução é a de calcular o tempo médio para a conclusão de uma solicitação e usar este valor como base de cálculo para determinar uma possível data de finalização. A utlização da média, neste caso, não deverá enviesar a distribuição de valores na coluna excessivamente.

In [5]:
(
    df[df['datahora_finalizacao_solicitacao'] .isna()]
    [['datahora_abertura_solicitacao', 'datahora_finalizacao_solicitacao']]
)

,datahora_abertura_solicitacao,datahora_finalizacao_solicitacao
50,2024-12-24 08:24:45,NaT
79,2024-12-20 15:54:27,NaT
126,2024-12-19 17:09:22,NaT
127,2024-12-19 17:09:22,NaT
162,2024-12-18 16:29:24,NaT
...,...,...
12707,2024-09-16 10:45:05,NaT
12710,2024-09-16 10:44:54,NaT
18024,2024-08-14 18:03:59,NaT
20833,2024-07-29 16:13:11,NaT


In [6]:
delta_med_fin = (df['datahora_finalizacao_solicitacao'] - df['datahora_abertura_solicitacao']).mean()

# O tempo médio para que uma solicitação seja correspondida é de praticamente 2 semanas

In [7]:
df['datahora_finalizacao_solicitacao'] = (
    df['datahora_finalizacao_solicitacao']
    .where(
        cond=pd.notna(df['datahora_finalizacao_solicitacao']),
        other=df['datahora_abertura_solicitacao'] + delta_med_fin)
)

In [8]:
## Algumas linhas preenchidas ilustram o processo realizado

(
    df
    [['datahora_abertura_solicitacao', 'datahora_finalizacao_solicitacao']]
    .iloc[[50, 79, 126, 127, 162]]
)

,datahora_abertura_solicitacao,datahora_finalizacao_solicitacao
50,2024-12-24 08:24:45,2025-01-07 03:31:21.111615942
79,2024-12-20 15:54:27,2025-01-03 11:01:03.111615942
126,2024-12-19 17:09:22,2025-01-02 12:15:58.111615942
127,2024-12-19 17:09:22,2025-01-02 12:15:58.111615942
162,2024-12-18 16:29:24,2025-01-01 11:36:00.111615942


### area_processo

O valor de Área do Processo, como varíavel categórica, e contendo somente 1 valor inválido, pode ser substuída com confiança pela moda da categoria.

In [9]:
df['area_processo'] = df['area_processo'].fillna(value=df['area_processo'].mode()[0])

### comarca

Os dados da comarca, por sua vez, possuem uma particularidade: O órgão não permite definir exatamente qual a jurisdição da audiência, e sem maiores informações a respeito do negócio, não é possível preencher estes dados com grande confiança.

Entretanto, como a UF da comarca está disponível, e é do Rio Grande do Sul, podemos utilizar esta informação para -- pelo menos -- preencher os dados vazios com a principal comarca do Estado.

In [10]:
(
    df[df['comarca'].isnull()]
    [['uf_comarca', 'comarca', 'orgao']]
)

,uf_comarca,comarca,orgao
1277,RS,NaN,Órgão integração API
5623,RS,NaN,Órgão integração API
6418,RS,NaN,Órgão integração API
7128,RS,NaN,Órgão integração API
26772,RS,NaN,Órgão integração API
27914,RS,NaN,Órgão integração API
28477,RS,NaN,Órgão integração API
29308,RS,NaN,Órgão integração API


In [11]:
main_com_rs = df[df['uf_comarca'] == 'RS']['comarca'].mode()
df['comarca'] = df['comarca'].fillna(value=df['comarca'].mode()[0])

### situacao_dados

Esta categoria, enfim, é particular porque os dados ausentes são provenientes de um teste realizado pela empresa e que foi extraído para a base de análise.

Como são dados que não representam a realidade do negócio na perspectiva desta análise, não deverão ser considerados e, portanto, excluídos.

In [12]:
df[df['situacao_dados'].isnull()][['orgao']]

,orgao
465,Órgão TESTE DOC9
14620,Órgão TESTE DOC9


In [13]:
df = df.drop(
    df[df['situacao_dados'].isnull()][['orgao']].index
)

Os dados devidamente tratados são salvos para serem usados nas análises seguintes.

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29998 entries, 0 to 29999
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   id_solicitacao                    29998 non-null  int64         
 1   nome_parceiro                     29998 non-null  object        
 2   nome_cliente                      29998 non-null  object        
 3   datahora_abertura_solicitacao     29998 non-null  datetime64[ns]
 4   datahora_audiencia                29998 non-null  datetime64[ns]
 5   prazo_para_inserir_dados          29998 non-null  datetime64[ns]
 6   datahora_finalizacao_solicitacao  29998 non-null  datetime64[ns]
 7   tipo                              29998 non-null  object        
 8   tipo_demanda                      29998 non-null  object        
 9   area_processo                     29998 non-null  object        
 10  tipo_audiencia                    29998 non-null  o

In [15]:
df.to_csv(data / 'limpo_solicitacoes_doc9.csv')